In [1]:
import numpy as np
import numpy as np
import pygmt as pg
import pygeoinf as inf
from pygeoinf.symmetric_space.sphere import Sobolev
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(7)

In [2]:
import pygeoinf

In [3]:
pg.config(MAP_FRAME_TYPE='plain')
pg.config(FONT_ANNOT_PRIMARY="10p,Palatino-Roman,black")
pg.config(FONT_ANNOT_SECONDARY="10p,Palatino-Roman,black")
pg.config(FONT_LABEL="10p,Palatino-Roman,black")

cmap = '/space/ij264/earth-tunya/cpts/vik_DT.cpt'

ROOT_PATH = '/space/ij264/earth-tunya/geoinf_analysis/figures/real_data'

In [4]:
# Read in data.
N_DATA = 1000
data = pd.read_csv(
    '/space/ij264/earth-tunya/geoinf_analysis/data/global.xyz',
    names=['lon', 'lat', 'z', 'z_err', 'symbol'],
    sep=r' '
).sample(N_DATA)

In [5]:
# Model space parameters
MODEL_LMAX = 64
MODEL_ORDER = 2.
MODEL_SCALE = 0.1
RADIUS = 1.

model_space = Sobolev(MODEL_LMAX, MODEL_ORDER, MODEL_SCALE, radius=RADIUS)

In [6]:
# Convert points to correct format.
selected_points = list(zip(data['lat'], data['lon']))

In [7]:
# Construct forward model.
forward_operator = model_space.point_evaluation_operator(selected_points)
data_error_measure = inf.GaussianMeasure.from_standard_deviations(forward_operator.codomain, data['z_err'].to_numpy())

forward_problem = inf.LinearForwardProblem(
    forward_operator,
    data_error_measure=data_error_measure
)

In [8]:
# Prior parameters.
PRIOR_ORDER = 2.
PRIOR_SCALE = 0.1

# Set the unconstrained prior
unconstrained_model_prior_measure = (
    model_space.point_value_scaled_sobolev_kernel_gaussian_measure(
        PRIOR_ORDER, PRIOR_SCALE
    )
)

# Setup constraint: zero mean value.
constraint_operator = model_space.to_coefficient_operator(0, lmin=0)
constraint_value = np.array([0])
constraint = inf.AffineSubspace.from_linear_equation(
    constraint_operator, constraint_value, solver=inf.CholeskySolver()
)

# Form the constrained prior
model_prior_measure = constraint.condition_gaussian_measure(
    unconstrained_model_prior_measure
)

In [9]:
# Calculate the preconditioner.
LMAX_PRECONDITIONER = 16
model_space_preconditioner = Sobolev(
    LMAX_PRECONDITIONER, MODEL_ORDER, MODEL_SCALE, radius=RADIUS)
forward_operator_preconditioner = model_space_preconditioner.point_evaluation_operator(selected_points)
forward_problem_preconditioner = inf.LinearForwardProblem(
    forward_operator_preconditioner,
    data_error_measure=data_error_measure
)

# Set the unconstrained prior
unconstrained_model_prior_measure_preconditioner = (
    model_space_preconditioner.point_value_scaled_sobolev_kernel_gaussian_measure(
        PRIOR_ORDER, PRIOR_SCALE
    )
)

# Setup constraint: zero mean value.
constraint_operator_preconditioner = model_space_preconditioner.to_coefficient_operator(0, lmin=0)
constraint_value = np.array([0])
constraint_preconditioner = inf.AffineSubspace.from_linear_equation(
    constraint_operator_preconditioner, constraint_value, solver=inf.CholeskySolver()
)

# Form the constrained prior
model_prior_measure_preconditioner = constraint_preconditioner.condition_gaussian_measure(
    unconstrained_model_prior_measure_preconditioner
)

In [10]:
# Inversion with preconditioner.
inversion_preconditioner = inf.LinearBayesianInversion(
    forward_problem_preconditioner, model_prior_measure_preconditioner)
normal_operator_preconditioner = inversion_preconditioner.normal_operator


In [11]:
# Form inverse using eigendecomposition.
print('Forming the preconditioner.')
solver = inf.EigenSolver(parallel=False)
inverse_normal_operator_preconditioner = solver(normal_operator_preconditioner)

Forming the preconditioner.


In [12]:
# Set up the Bayesian inversion method
bayesian_inversion = inf.LinearBayesianInversion(forward_problem, model_prior_measure)

# Solve for the posterior distribution
print("Solving the linear system...")
model_posterior_measure = bayesian_inversion.model_posterior_measure(
    data['z'], inf.CGMatrixSolver(), preconditioner=inverse_normal_operator_preconditioner
)
print("Done.")
# Get the posterior expectation
model_posterior_expectation = model_posterior_measure.expectation

Solving the linear system...
Done.


In [13]:
fig = pg.Figure()
fig.basemap(region='d', projection="H12c", frame='f')
fig.grdimage(
    model_posterior_expectation.to_xarray(),
    cmap=cmap,
    dpi=150
)
fig.coast(shorelines=True, area_thresh=1e5, frame='f')
# fig.plot(
#     x=data['lon'],
#     y=data['lat'],
#     style='c0.1c',
#     pen='1p,black'
# )
fig.colorbar(position='+h+e', frame='af+lPosterior mean (m)')
fig.savefig(f'{ROOT_PATH}/{N_DATA}_points_posterior_mean_with_preconditioner.png')

In [18]:
power = model_space._sample_power_measure(model_posterior_measure, 10, parallel=True, n_jobs=8)

Backend threading restricted to 1 thread(s).
Backend threading restricted to -1 thread(s).


In [15]:
fig = pg.Figure()
fig.basemap(region=[0, MODEL_LMAX, 1e-3, 1e1], projection='X10c/5cl', frame=['af', 'WSne', 'y+lPosterior Mean Power Spectrum', 'x+lSpherical harmonic degree'])
fig.plot(
    x=np.arange(MODEL_LMAX + 1),
    y=model_posterior_measure.power(),
    pen='1p,black'
)
fig.savefig(f'{ROOT_PATH}/{N_DATA}_points_posterior_mean_power_spectrum_with_preconditioner.png')

AttributeError: 'GaussianMeasure' object has no attribute 'power'